In [1]:
!pip install -q -r ../requirements.txt

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score, accuracy_score
from sklearn.inspection import permutation_importance
import shap
from tqdm import tqdm
import warnings
import joblib
import gc

In [ ]:
sys.path.append("../src")
from utils import *

In [ ]:
warnings.simplefilter("ignore")

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df_attrition = pd.read_csv("../cortAIx Factory Data Science Technical Test project #1.csv")
df_attrition.head()

In [ ]:
df_attrition=df_attrition.iloc[:, 2:]
df_attrition.head()

In [ ]:
df_attrition.shape

In [ ]:
df_attrition.info()

In [ ]:
df_attrition.describe()

In [ ]:
df_attrition.head()

# Statistics

## Categorical Data

In [ ]:
df_attrition["Gender"].value_counts(normalize=True, dropna=False)

In [ ]:
df_attrition["Department"].value_counts(normalize=True, dropna=False)

In [ ]:
df_attrition["Job_Title"].value_counts(normalize=True, dropna=False)

In [ ]:
df_attrition["Office_Localisation"].value_counts(normalize=True, dropna=False)

In [ ]:
df_attrition["Promotion_Last_5Years"].value_counts(normalize=True, dropna=False)

Most categorical data are uniformly distributed, with some of the following issues:
- Missing values in Department
- Missing values in Job_Title
- Outliers in Job_Title (values 1 and 2)
- Outliers in Promotion_Last_5Years (values 45124 and 32770)

In [ ]:
mask_categorical = (
    df_attrition["Department"].isnull() |
    df_attrition["Job_Title"].isnull() |
    df_attrition["Job_Title"].isin(["1", "2"]) |
    df_attrition["Promotion_Last_5Years"].isin([45124, 32770])
)

## Numerical Data

In [ ]:
# We check for missing values
df_attrition[["Age", "Years_at_Company",
             "Satisfaction_Level", "Average_Monthly_Hours",
             "Salary"]].isnull().sum()

In [ ]:
# We check for not consistent data
df_attrition[df_attrition["Age"] < df_attrition["Years_at_Company"]]

In [ ]:
# We also check for 0 salaries to look for outliers
df_attrition[df_attrition["Salary"]==0]

In [ ]:
# We check for outliers using boxplot
fig, axes = plt.subplots(2, 3, figsize=(20, 5))
axes[1, 2].remove()
axes = axes.flatten()

for i, col in enumerate(["Age", "Years_at_Company", "Satisfaction_Level",
                        "Average_Monthly_Hours", "Salary"]):
    plt.sca(axes[i])
    sns.boxplot(y=df_attrition[col])

- Variables Years_at_Company and Satisfaction_Level present some outliers as seen in the boxplots respectively for over 90 and over 150
- Salaries equal to 0 are for employees being in the company for more than 3 years, which is weird

In [ ]:
mask_numerical = (
    df_attrition["Age"].isnull() |
    df_attrition["Years_at_Company"].isnull() |
    (df_attrition["Age"] < df_attrition["Years_at_Company"]) |
    (df_attrition["Years_at_Company"] > 90) |
    (df_attrition["Satisfaction_Level"] > 150) |
    (df_attrition["Salary"] == 0)
)

## Target

In [ ]:
df_attrition["Attrition"].value_counts(normalize=True, dropna=False)

The data is not imbalanced, we will not need to use techniques like SMOTE or ADASYN, or change the losses of the classification algorithms.

However, the target has some missing values.

In [ ]:
mask_target = df_attrition["Attrition"].isnull()

## Dealing with missing values and outliers

In [ ]:
len(df_attrition[mask_categorical | mask_numerical | mask_target])/len(df_attrition)

In [ ]:
# We drop rows with missing values as they only reprsent 1% of the dataset
df_attrition = df_attrition[~(mask_categorical | mask_numerical | mask_target)]

# We can also drop the Office_Localisation column as it only has one category
df_attrition.drop(columns="Office_Localisation", inplace = True)

df_attrition.head()

## Data Preprocessing

In [ ]:
# We fix data types
df_attrition.dtypes

In [ ]:
df_attrition["Average_Monthly_Hours"].unique()

In [ ]:
df_attrition["Average_Monthly_Hours"]=df_attrition["Average_Monthly_Hours"].apply(lambda x: int(keep_only_digits(x)))

In [ ]:
df_attrition["Average_Monthly_Hours"].unique()

In [ ]:
df_attrition["Attrition"]=df_attrition["Attrition"].apply(int)

In [ ]:
df_attrition.head()

In [ ]:
categorical_columns = ["Gender", "Department", "Job_Title", "Promotion_Last_5Years"]
numerical_columns = ["Age", "Years_at_Company", "Satisfaction_Level", "Average_Monthly_Hours", "Salary"]
target_column = ["Attrition"]

## Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 8))
axes = axes.flatten()

for i, col in enumerate(categorical_columns):
    plt.sca(axes[i])
    sns.countplot(data=df_attrition,
                 x=col,
                 hue="Attrition")

In [ ]:
_ = df_attrition[numerical_columns].hist(figsize=(10, 8), bins=30)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 5))
axes[1,2].remove()
axes = axes.flatten()

for i, col in enumerate(numerical_columns):
    plt.sca(axes[i])
    sns.kdeplot(data=df_attrition,
                 x=col,
                 hue="Attrition")

## Correlations

In [ ]:
_ = sns.heatmap(df_attrition[numerical_columns + target_column].corr(),
               annot=True,
               cmap="Reds",
               fmt=".2f")

## Feature Engineering

### Numerical Variables binning

In [ ]:
# We discretize Age variables into age bins
bins = [0, 18, 30, 40, 50, 60, 100]
labels = [1, 2, 3, 4, 5, 6]

df_attrition["Age_bin"]=pd.cut(df_attrition["Age"], bins=bins, labels=labels, right=False)

_=sns.countplot(data=df_attrition,
                 x="Age_bin",
                 hue="Attrition")

In [ ]:
# We discretize Salary variable by bins of 10000
bins = list(range(30000, 110000, 10000))
labels = [i+1 for i in range(len(bins)-1)]

df_attrition["Salary_bin"]=pd.cut(df_attrition["Salary"], bins=bins, labels=labels, right=False)

_=sns.countplot(data=df_attrition,
                 x="Salary_bin",
                 hue="Attrition")

In [ ]:
# We discretize Satisfaction_Level variable 
bins = [0, .25, .5, .75, 1]
labels = [1, 2, 3, 4]

df_attrition["Satisfaction_Level_bin"]=pd.cut(df_attrition["Satisfaction_Level"], bins=bins, labels=labels, right=False)

_=sns.countplot(data=df_attrition,
                 x="Satisfaction_Level_bin",
                 hue="Attrition")

### Categorical variables interaction

In [ ]:
df_attrition["Department x Job_Title"] = df_attrition["Department"] + " - " + df_attrition["Job_Title"]
df_attrition.sort_values(by = "Department x Job_Title", ascending = True, inplace = True)
_ =  sns.countplot(data=df_attrition,
                   x="Department x Job_Title",
                   hue="Attrition")
_ = plt.xticks(rotation=90)

This interaction term brings the same information as a breakdown by Department, it seems like the Department is more important for attrition than job title and interaction term.

## Feature Selection

In [ ]:
# We check if binned features have a better predictive power or not
for col in tqdm(["Age", "Satisfaction_Level", "Salary"]):

    model = RandomForestClassifier()

    X_train, X_test, y_train, y_test = train_test_split(df_attrition[[f"{col}", f"{col}_bin"]],
                                                       df_attrition["Attrition"],
                                                       test_size=.2,
                                                       random_state=42)
    
    model.fit(X_train, y_train)
    
    # Compute importance
    importance = permutation_importance(model, X_test, y_test, scoring="recall", n_repeats=100, random_state=42)
    
    # Print feature importance
    feature_importance = pd.DataFrame(
        {"Feature": [f"{col}", f"{col}_bin"], "Importance": importance.importances_mean}
    ).sort_values(by="Importance", ascending=False)

    print(feature_importance)

Binned versions of features do not improve the model

In [ ]:
# We check if the interaction term between Department and Job Title 
# has a higher feature importance
preprocessor = ColumnTransformer(
    transformers = [
        ("target_encoder", TargetEncoder(), ["Department", "Job_Title", "Department x Job_Title"])
    ]
)
model = RandomForestClassifier()

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", model)
])

X_train, X_test, y_train, y_test = train_test_split(df_attrition[["Department", "Job_Title", "Department x Job_Title"]],
                                                   df_attrition["Attrition"],
                                                   test_size=.2,
                                                   random_state=42)

pipeline.fit(X_train, y_train)

# Compute importance
importance = permutation_importance(pipeline, X_test, y_test, scoring="recall", n_repeats=100, random_state=42)

# Print feature importance
feature_importance = pd.DataFrame(
    {"Feature": ["Department", "Job_Title", "Department x Job_Title"], 
     "Importance": importance.importances_mean}
).sort_values(by="Importance", ascending=False)

print(feature_importance)

In [ ]:
df_attrition.drop(columns=["Age_bin", "Salary_bin", "Satisfaction_Level_bin", "Department x Job_Title", "Job_Title"], inplace=True)

# Classification pipeline

## Model Selection by Cross Validation

- For Gender variable we use One-Hot Encoding
- For Department and Job_Title we use Target Encoding as those variables present several categories
- We use normalization on numerical features and taregt encoded features, but not one binary features (Gender after One-Hot Encoding and Promotion_Last_5Years)
- Label Encoding and Normalization are made inside the Cross Validation for each train-test split to avoid data leakage
- We compare models on the recall as the recall is the metrics of interest here : we want to minimize the number of False Negatives as we don't want to predict people who will leave as people who want to stay, because in the case of someone leaving without predicting it the cost is high for the company
- Promotion_Last_5Years variable is already One Hot Encoded, it is not modified then

In [ ]:
# We create first a pipeline that target encodes Department and Job_Title
# variables and then scales the numerical variables created
pipeline_target_encoder = make_pipeline(
    TargetEncoder(),
    StandardScaler()
)

# We create a preprocessing pipeline for all variables to use in Cross Validation
preprocessor = ColumnTransformer(
    transformers = [
        ("scaler", StandardScaler(), ["Age", "Years_at_Company", "Average_Monthly_Hours", "Salary"]),
        ("target_encoder", pipeline_target_encoder, ["Department"]),
        ("one_hot_encoder", OneHotEncoder(drop="first", handle_unknown="ignore"), ["Gender"]),
        ("passthrough", "passthrough", ["Promotion_Last_5Years", "Satisfaction_Level"])
    ]
)

# We define classifiers
dict_classifiers = {
    "Logistic Regression":LogisticRegression(),
    "Ridge Regression":LogisticRegression(penalty='l2', solver='lbfgs', C=1.0),
    "Lasso Regression":LogisticRegression(penalty='l1', solver='liblinear', C=1.0),
    "ElasticNet Regression":LogisticRegression(penalty='elasticnet', solver='saga', C=1.0, l1_ratio=0.5),
    "Random Forest":RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
    "Linear SVM": SVC(kernel="linear", probability=True),
    "RBF SVM":SVC(kernel="rbf", probability=True),
    "Gradient Boosting":GradientBoostingClassifier(),
    "XGB":XGBClassifier(),
    "LGBM":LGBMClassifier(verbose=-1),
    "MLP":MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500),
    "Gaussian Naive Bayes":GaussianNB(),
    "Bernoulli Naive Bayes":BernoulliNB(),
    "Bagging":BaggingClassifier(),
    "Voting":VotingClassifier(estimators=[('rf', RandomForestClassifier()), ('xgb', XGBClassifier())], voting='soft'),
    "KNN":KNeighborsClassifier(n_neighbors=5),
    "Quadratic Discriminant Analysis":QuadraticDiscriminantAnalysis()
}

dict_recalls = {}

for name, classifier in tqdm(dict_classifiers.items()):
    
    # We create the final pipeline (preprocessing + classifier)
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", classifier)
    ])
    
    # We apply cross validation and compute mean recall
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    scores = cross_val_score(pipeline,
                            df_attrition.drop(columns=["Attrition"]),
                            df_attrition["Attrition"],
                            cv=cv,
                            scoring="recall")

    dict_recalls[name]=float(np.mean(scores))

In [ ]:
dict(sorted(dict_recalls.items(), key=lambda item: item[1]))

Linear SVM gives the best score, we will fine tune it.

## Fine tuning of the model

In [ ]:
# We create first a pipeline that target encodes Department and Job_Title variables and then scales the numerical variables created
pipeline_target_encoder = make_pipeline(
    TargetEncoder(),
    StandardScaler()
)

# We create a preprocessing pipeline for all variables to use in Cross Validation
preprocessor = ColumnTransformer(
    transformers = [
        ("scaler", StandardScaler(), ["Age", "Years_at_Company", "Average_Monthly_Hours", "Salary"]),
        ("target_encoder", pipeline_target_encoder, ["Department"]),
        ("one_hot_encoder", OneHotEncoder(drop="first", handle_unknown="ignore"), ["Gender"]),
        ("passthrough", "passthrough", ["Promotion_Last_5Years", "Satisfaction_Level"])
    ]
)

# Define the Linear SVM model
svm_model = SVC(kernel="linear", probability=True, random_state=42)

# Define the parameter grid for tuning SVM
param_grid = {
    'classifier__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'classifier__max_iter': [-1]  # Maximum number of iterations for convergence
}

# Create the final pipeline (preprocessing + classifier)
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", svm_model)
])

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(n_splits=5, shuffle=True), scoring="recall")

# Fit the model with grid search
grid_search.fit(df_attrition.drop(columns=["Attrition"]), df_attrition["Attrition"])

# Print the best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Recall Score: {grid_search.best_score_:.4f}")

# Optionally, you can retrieve the best estimator and use it for prediction:
best_svm_model = grid_search.best_estimator_

Best model is linear SVM with c=0.01

## Model evaluation

In [ ]:
best_svm_model

In [ ]:
# We train test split the data then use the pipeline (preprocessors + linear SVM with C=0.1)
# found earlier and evaluate the model
X_train, X_test, y_train, y_test = train_test_split(df_attrition.drop(columns="Attrition"),
                                                   df_attrition["Attrition"],
                                                   test_size=.2,
                                                   random_state=42)


best_svm_model.fit(X_train, y_train)
y_pred = best_svm_model.predict(X_test)

In [ ]:
# We make the evaluation
print(classification_report(y_test, y_pred))
print("\n")
print(confusion_matrix(y_test, y_pred))
print("\n")
print(f"Recall score : {recall_score(y_test, y_pred):.2f}")
print("\n")
print(f"Accuracy score : {accuracy_score(y_test, y_pred):.2f}")

## Feature importance

We use permutation feature importance and SHAP, two feature importance measures that are model agnostic

### Permutation Feature Importance

In [ ]:
# Compute permutation importance
perm_importance = permutation_importance(best_svm_model, 
                                         X_test, 
                                         y_test, 
                                         scoring="recall", 
                                         n_repeats=10, 
                                         random_state=42)

# Create DataFrame
perm_importance_df = pd.DataFrame({
    "Feature": X_test.columns,
    "Importance": perm_importance.importances_mean
}).sort_values(by="Importance", ascending=False)

perm_importance_df.set_index("Feature").plot(kind="bar")
_ = plt.axhline(0, color='black', linewidth=.5)

### SHAP

In [ ]:
# Get the preprocessor step from the pipeline and transform X_test
preprocessor = best_svm_model.named_steps["preprocessor"]
X_test_transformed = preprocessor.transform(X_test)

# Get feature names after preprocessing
feature_names = best_svm_model.named_steps["preprocessor"].get_feature_names_out()

# Use KernelExplainer for SHAP (since SVM is not tree-based)
explainer = shap.KernelExplainer(best_svm_model.named_steps["classifier"].predict_proba, X_test_transformed)

# Compute SHAP values
shap_values = explainer.shap_values(X_test_transformed)

In [ ]:
shap_values.shape

In [ ]:
shap_values.shape[1]

In [ ]:
# Convert SHAP values to a DataFrame for plotting
shap_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Shapley Importance": np.mean(np.abs(shap_values[..., 1]), axis=0)  # Absolute importance for positive class
}).sort_values(by="Shapley Importance", ascending=False)

# Plot the feature importance
shap_importance_df.set_index("Feature").plot(kind="bar", figsize=(10, 5))
plt.axhline(0, color="black", linewidth=0.5)
plt.title("Feature Importance using SHAP")
plt.show()

In [ ]:
# SHAP Summary Plot
shap.summary_plot(shap_values[..., 1], X_test_transformed, feature_names=feature_names)

## Model saving

In [ ]:
# Save the model
joblib.dump(best_svm_model, "attrition_classification_model.pkl")